# Access MLflow UI via APIs

In [1]:
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType
import mlflow
from datetime import datetime

## Get experiements

In [24]:
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [25]:
client.search_experiments()

[<Experiment: artifact_location='/Users/yihanzhou/PycharmProjects/mlops-zoomcamp/00-my-work/wk2/mlruns/2', creation_time=1716419576909, experiment_id='2', last_update_time=1716419576909, lifecycle_stage='active', name='demo_experiment', tags={}>,
 <Experiment: artifact_location='/Users/yihanzhou/PycharmProjects/mlops-zoomcamp/00-my-work/mlruns/1', creation_time=1716360482562, experiment_id='1', last_update_time=1716360482562, lifecycle_stage='active', name='nyc_taxis', tags={}>]

## Create new experiment

In [5]:
client.create_experiment(name="demo_experiment")

In [6]:
runs = client.search_runs(
    experiment_ids='1',
    filter_string='metrics.rmse < 6.8',
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=['metrics.rmse ASC']
)

In [7]:
for r in runs:
    print(f"run id: {r.info.run_id}, rmse: {r.data.metrics['rmse']:.4f}")

run id: 8aa8ce40637e49a99803c419b8d1149d, rmse: 4.9903
run id: 9d3949c3f9944fdfa92893a9f443f2c7, rmse: 5.0029
run id: 0dd19feb122a48efb456798058d83949, rmse: 5.0042
run id: 4b27d705160442e68f18a0d8b67c192b, rmse: 5.0055
run id: f170df1d58e84d01a6f64c1951a196ff, rmse: 5.0098


# Promote models for registry

In [28]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [29]:
run_id = "b9078223b5944af9ac8a0a6c57127cdb"
model_uri = f"runs:/{run_id}/model"
name = "nyc_taxis_xgboost"
mlflow.register_model(model_uri=model_uri, name=name)

Registered model 'nyc_taxis_xgboost' already exists. Creating a new version of this model...
Created version '3' of model 'nyc_taxis_xgboost'.


<ModelVersion: aliases=[], creation_timestamp=1716422952472, current_stage='None', description=None, last_updated_timestamp=1716422952472, name='nyc_taxis_xgboost', run_id='b9078223b5944af9ac8a0a6c57127cdb', run_link=None, source='/Users/yihanzhou/PycharmProjects/mlops-zoomcamp/00-my-work/mlruns/1/b9078223b5944af9ac8a0a6c57127cdb/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

## Get latest versions

Registered model 'nyc_taxis_xgboost' already exists. Creating a new version of this model...
Created version '2' of model 'nyc_taxis_xgboost'.

This will reflect on mlflow ui that the Latest version is "Version 2"

In [ ]:
latest_versions = client.get_latest_versions(name=name)
for v in latest_versions:
    print(f"version: {v.version}, stage: {v.current_stage}")

## Update model stage
Assuming we want to transit "version 2" to "Staging". 

In [ ]:
client.transition_model_version_stage(
    name=name,
    version=2,
    stage="Staging",
    archive_existing_versions=False
)

## Annotate model

In [ ]:
date = datetime.today().date()
client.update_model_version(
    name=name,
    version=2,
    description=f"The model version was transitioned to staging on {date}."
)

## Compare versions 

In [9]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(path):
    df = pd.read_parquet(path)
    df['duration'] = df['lpep_dropoff_datetime'] - df['lpep_pickup_datetime']
    df['duration'] = df['duration'].apply(lambda x: x.total_seconds()/60)
    df = df[(df['duration'] > 1.0) & (df['duration'] < 60.0)]
    categorical = ['PULocationID','DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df

def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [11]:
data_2403_path = "/Users/yihanzhou/PycharmProjects/mlops-zoomcamp/data/green_tripdata_2024-03.parquet"
df = read_dataframe(data_2403_path)

In [16]:
client.download_artifacts(run_id=run_id, path="preprocessor", dst_path=".")

'/Users/yihanzhou/PycharmProjects/mlops-zoomcamp/00-my-work/wk2/preprocessor'

In [17]:
import pickle

with open("preprocessor/preprocessor.b","rb") as f_in:
    dv = pickle.load(f_in)

In [19]:
x_test = preprocess(df, dv)

In [20]:
target = "duration"
y_test = df[target].values

In [31]:
%time test_model(name=name, stage="Production", X_test=x_test, y_test=y_test)

/opt/anaconda3/envs/mlopszoomcamp/lib/python3.12/site-packages/mlflow/store/artifact/utils/models.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.12.2/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])
/opt/anaconda3/envs/mlopszoomcamp/lib/python3.12/site-packages/xgboost/core.py:160: UserWarning: [17:09:28] WARNING: /var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_b6b50u1t0e/croot/xgboost-split_1713972723244/work/cpp_src/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


CPU times: user 2.3 s, sys: 112 ms, total: 2.41 s
Wall time: 238 ms


/opt/anaconda3/envs/mlopszoomcamp/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


{'rmse': 5.754326871459331}

In [32]:
%time test_model(name=name, stage="Staging", X_test=x_test, y_test=y_test)

/opt/anaconda3/envs/mlopszoomcamp/lib/python3.12/site-packages/mlflow/store/artifact/utils/models.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.12.2/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])
/opt/anaconda3/envs/mlopszoomcamp/lib/python3.12/site-packages/xgboost/core.py:160: UserWarning: [17:09:51] WARNING: /var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_b6b50u1t0e/croot/xgboost-split_1713972723244/work/cpp_src/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


CPU times: user 2.21 s, sys: 47.5 ms, total: 2.26 s
Wall time: 218 ms


/opt/anaconda3/envs/mlopszoomcamp/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


{'rmse': 5.754326871459331}